### 利用 Numba 加速 Python 代码

#### 例1：加速循环

这是一个采用插入排序算法来对 10000 个整数进行排序的例子

In [5]:
import time
import random

num_loops = 50
len_of_list = 100000

def insertion_sort(arr):
    for i in range(len(arr)):
        cursor = arr[i]
        pos = i
        
        while pos > 0 and arr[pos-1] > cursor:
            # 从后往前对比，从小到大排序
            arr[pos] = arr[pos-1]
            pos = pos-1
        # 找到当前元素的位置
        arr[pos] = cursor
    return arr

In [6]:
start = time.time()
list_of_numbers = list()
for i in range(len_of_list):
    num = random.randint(0, len_of_list)
    list_of_numbers.append(num)

for i in range(num_loops):
    result = insertion_sort(list_of_numbers)

end = time.time()

run_time = end-start
print('Average time={}'.format(run_time/num_loops))

Average time=22.84399790763855


添加两行代码，来使用 Numba 加速代码

In [8]:
import time
import random
from numba import jit

num_loops = 50
len_of_list = 100000

@jit(nopython=True)
def insertion_sort(arr):
    for i in range(len(arr)):
        cursor = arr[i]
        pos = i
        
        while pos > 0 and arr[pos-1] > cursor:
            # 从后往前对比，从小到大排序
            arr[pos] = arr[pos-1]
            pos = pos-1
        # 找到当前元素的位置
        arr[pos] = cursor
    return arr

In [9]:
start = time.time()
list_of_numbers = list()
for i in range(len_of_list):
    num = random.randint(0, len_of_list)
    list_of_numbers.append(num)

for i in range(num_loops):
    result = insertion_sort(list_of_numbers)

end = time.time()

run_time = end-start
print('Average time={}'.format(run_time/num_loops))

Average time=0.09438572406768798


#### 例2：加速 Numpy 运算

第二个例子就是对 Numpy 运算的加速，这里创建几个非常大的数组进行运算

In [10]:
import time
import numpy as np

num_loops = 50
img1 = np.ones((1000, 1000), np.int64) * 5
img2 = np.ones((1000, 1000), np.int64) * 10
img3 = np.ones((1000, 1000), np.int64) * 15

def add_arrays(img1, img2, img3):
    return np.square(img1+img2+img3)

In [13]:
start1 = time.time()
for i in range(num_loops):
    result = add_arrays(img1, img2, img3)
end1 = time.time()
run_time1 = end1 - start1
print('Average time for normal numpy operation={}'.format(run_time1/num_loops))

Average time for normal numpy operation=0.040156774520874024


In [14]:
# numba 加速
from numba import vectorize, int64

@vectorize([int64(int64,int64,int64)], target='parallel')
def add_arrays_numba(img1, img2, img3):
    return np.square(img1+img2+img3)

In [15]:
start2 = time.time()
for i in range(num_loops):
    result = add_arrays_numba(img1, img2, img3)
end2 = time.time()
run_time2 = end2 - start2
print('Average time using numba accelerating={}'.format(run_time2/num_loops))

Average time using numba accelerating=0.007735490798950195
